In [6]:
from google.colab import drive
import os
from pathlib import Path
import math
import numpy as np
from PIL import Image
import tensorflow as tf
import albumentations as A
import splitfolders
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import random

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### **Аугментация**

In [9]:
## изменение изображения
def editImage(image):
  transform = A.Compose([
        A.Rotate(limit=60),
        A.Flip(),
        A.Transpose(),
        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 20.0)),
        ], p=0.2),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50,
                          rotate_limit=45, p=.75),
        # A.GridDistortion(p=.1),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
            A.PiecewiseAffine(p=0.3),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.Sharpen(),
            A.Emboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    ])
  random.seed(42)
  augmented_image = transform(image=image)['image']
  return augmented_image

In [10]:
## добавляем в папку новые измененные изображения
def folderChange(name):
  folder = os.listdir(name)
  n = len(folder)
  numDup = math.ceil((100 - n) / n)
  for i in folder:     
    image = cv2.imread(name + '/' + i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    for j in range(numDup):
      newimage = editImage(image)
      cv2.imwrite(os.path.join(name , str(j) + '-' + i), newimage)
  return 

In [11]:
path = 'gdrive/My Drive/ТП//dataset2'
## проделываем это с каждой папкой
for address, dirs, files in os.walk(path):
  if files:
     folderChange(address)

/usr/local/lib/python3.8/dist-packages/albumentations/imgaug/transforms.py:233: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.8/dist-packages/albumentations/imgaug/transforms.py:176: FutureWarning: This augmentation is deprecated. Please use Emboss instead
  warnings.warn("This augmentation is deprecated. Please use Emboss instead", FutureWarning)


## **Обучение**

In [ ]:
## добавляем черные границы если изображение слишком маленькое
def pad(img, shape):
    color_pick = img[0][0]
    padded_img = color_pick * np.ones(shape + img.shape[2:3], dtype=np.uint8)
    x_offset = int((padded_img.shape[0] - img.shape[0]) / 2)
    y_offset = int((padded_img.shape[1] - img.shape[1]) / 2)
    padded_img[x_offset:x_offset + img.shape[0], y_offset:y_offset + img.shape[1]] = img
    return padded_img

In [8]:
## изменяем размер изображения
def resize(img, shape):
    scale = min(shape[0] * 1.0 / img.shape[0], shape[1] * 1.0 / img.shape[1])
    if scale != 1:
        img = cv2.resize(img, dsize=None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    return img

In [24]:
##нумеруем классы для папок
filenames = []
labels = []
path = 'gdrive/My Drive/ТП//dataset2'
for idx, class_dir in enumerate(os.listdir(path)):
  for file in os.listdir(os.path.join(path, class_dir)):
    filenames.append(os.path.join(path, class_dir, file))
    labels.append(idx)

In [14]:
## делим на test и train 
train_filenames, test_filenames, train_labels, test_labels = train_test_split(filenames, labels, test_size=0.2, random_state=42)

In [15]:
## класс для обработки датасета
class AnimalDataset(Dataset):
  def __init__(self, filenames, labels):
    self._filenames = filenames
    self._labels = labels

  def __len__(self):
    return len(self._filenames)

  def __getitem__(self, idx):
    filename = self._filenames[idx]
    label = self._labels[idx]
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = resize(img, (224, 224))
    img = pad(img, (224, 224))
    img = torch.tensor(img, dtype=torch.float).permute(2, 0, 1) / 255.
    return img, label

In [16]:
## упорядочиваем данные 
train_dataset = AnimalDataset(train_filenames, train_labels)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=0)
test_dataset = AnimalDataset(test_filenames, test_labels)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=108, num_workers=0)

In [21]:
## для обучения используем resnet50 меняя последний слой на 3-ех классовый
model = resnet50(pretrained=True)
for param in model.parameters():
  param.requires_grad=False

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 3)
model.to('cuda')

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
## для тестирования
def testEpoch(model, epoch, test_loader):
    model.eval()
    with torch.no_grad():
      test_accuracy = []
      test_real = []
      for batch_x, batch_y in tqdm(test_loader):
          outputs = model(batch_x.to('cuda')).detach().cpu().numpy()
          test_accuracy.append(outputs)
          test_real.append(batch_y.detach().cpu().numpy())
      print("Epoch", epoch, "test accuracy", accuracy_score(np.hstack(test_real), np.argmax(np.hstack(test_accuracy), axis=1)))
    model.train()

In [23]:
## обучение на 25 эпох
ct = 0
for child in model.children():
  ct += 1
  if ct < 47:
    for param in child.parameters():
      param.requires_grad = True

for epoch in tqdm(range(25)):
  for batch in train_dataloader:
    optimizer.zero_grad()
    image, label = batch
    image = image.to('cuda')
    label = label.to('cuda')
    label_pred = model(image)
    loss = criterion(label_pred, label)
    loss.backward()
    optimizer.step()

  testEpoch(model, epoch, test_dataloader)

  4%|▍         | 1/25 [00:09<03:43,  9.33s/it]

Epoch 0 test accuracy 0.6



  8%|▊         | 2/25 [00:15<02:57,  7.72s/it]

Epoch 1 test accuracy 0.8



 12%|█▏        | 3/25 [00:23<02:48,  7.64s/it]

...

Epoch 22 test accuracy 0.9166666666666666



 96%|█████████▌| 24/25 [02:55<00:07,  7.08s/it]

Epoch 23 test accuracy 0.9333333333333333



100%|██████████| 25/25 [03:03<00:00,  7.33s/it]

Epoch 24 test accuracy 0.9
